In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
dataset_path = '/content/drive/MyDrive/<your_dataset_path>/cv-corpus-16.1-delta-2023-12-06-en.tar'


In [ ]:
!tar -xvf '/content/drive/MyDrive/<your_dataset_path>/cv-corpus-16.1-delta-2023-12-06-en.tar' -C '/content/'


tar: /content/drive/MyDrive/<your_dataset_path>/cv-corpus-16.1-delta-2023-12-06-en.tar: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
!pip install transformers torchaudio numpy

from transformers import Wav2Vec2Model
import torch
import torchaudio
import numpy as np
import os

# Load the pre-trained Wav2Vec 2.0 model
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h").to('cpu')

def preprocess_audio(file_path, chunk_size=16000*30):  # 30 seconds chunk
    waveform, sample_rate = torchaudio.load(file_path)
    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
        waveform = resampler(waveform)
    waveform = waveform.squeeze()
    return torch.split(waveform, chunk_size)

def extract_features(model, waveform):
    with torch.no_grad():
        outputs = model(waveform.unsqueeze(0))
    return outputs.last_hidden_state.squeeze(0)

# Set the directory path to your audio files in Google Drive
dataset_directory = '/content/drive/MyDrive/common voices/'

# List all audio files in the directory
all_files = [os.path.join(dataset_directory, f) for f in os.listdir(dataset_directory) if f.endswith('.mp3')]

# Process the files in batches of 10
batch_size = 10
for i in range(0, len(all_files), batch_size):
    batch_files = all_files[i:i+batch_size]
    for file in batch_files:
        # Preprocess the audio into chunks
        chunks = preprocess_audio(file)

        all_features = []
        for chunk in chunks:
            # Check if chunk is empty
            if chunk.numel() == 0:
                continue

            # Extract features
            features = extract_features(model, chunk)
            all_features.append(features.cpu().numpy())

            # Clear memory
            del chunk, features
            torch.cuda.empty_cache()

        # Concatenate all features from chunks and save to a file
        all_features = np.concatenate(all_features, axis=0)
        output_file = file.replace('.mp3', '_features.npy')
        np.save(output_file, all_features)

        print(f"Processed and saved features for {file}")

    print(f"Completed processing batch {i//batch_size + 1}/{len(all_files)//batch_size}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processed and saved features for /content/drive/MyDrive/common voices/common_voice_fr_38487441.mp3
Processed and saved features for /content/drive/MyDrive/common voices/common_voice_fr_38487439.mp3
Processed and saved features for /content/drive/MyDrive/common voices/common_voice_fr_38487440.mp3
Processed and saved features for /content/drive/MyDrive/common voices/common_voice_fr_38487438.mp3
Processed and saved features for /content/drive/MyDrive/common voices/common_voice_fr_38489943.mp3
Processed and saved features for /content/drive/MyDrive/common voices/common_voice_fr_38487451.mp3
Processed and saved features for /content/drive/MyDrive/common voices/common_voice_fr_38489941.mp3
Processed and saved features for /content/drive/MyDrive/common voices/common_voice_fr_38487452.mp3
Processed and saved features for /content/drive/MyDrive/common voices/common_voice_fr_38489942.mp3
Processed and saved features for /content/drive/MyDrive/common voices/common_voice_fr_38489944.mp3
Completed 

In [ ]:
!pip install transformers torchaudio numpy

from transformers import Wav2Vec2Model
import torch
import torchaudio
import numpy as np
import os

# Load the pre-trained Wav2Vec 2.0 model
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h").to('cpu')

def preprocess_audio(file_path, chunk_size=16000*30):  # 30 seconds chunk
    waveform, sample_rate = torchaudio.load(file_path)
    if sample_rate != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)
        waveform = resampler(waveform)
    waveform = waveform.squeeze()
    return torch.split(waveform, chunk_size)

def extract_features(model, waveform):
    with torch.no_grad():
        outputs = model(waveform.unsqueeze(0))
    return outputs.last_hidden_state.squeeze(0)

# Paths for processing and saving
process_directory = '/content/drive/MyDrive/comun voces fr/cv-corpus-16.1-delta-2023-12-06/fr/clips'
output_directory = '/content/drive/MyDrive/saved french npy'
os.makedirs(output_directory, exist_ok=True)

# List all audio files in the directory
all_files = [os.path.join(process_directory, f) for f in os.listdir(process_directory) if f.endswith('.mp3')]

# Process the files in batches of 10
batch_size = 10
for i in range(0, len(all_files), batch_size):
    batch_files = all_files[i:i+batch_size]
    concatenated_features = []
    for file in batch_files:
        # Preprocess the audio into chunks
        chunks = preprocess_audio(file)

        all_features = []
        for chunk in chunks:
            # Check if chunk is empty
            if chunk.numel() == 0:
                continue

            # Extract features
            features = extract_features(model, chunk)
            all_features.append(features.cpu().numpy())

            # Clear memory
            del chunk, features
            torch.cuda.empty_cache()

        # Concatenate features from each file
        concatenated_features.append(np.concatenate(all_features, axis=0))

    # Save the concatenated features for this batch
    batch_output_file = os.path.join(output_directory, f'batch_{i//batch_size + 1}_features.npy')
    np.save(batch_output_file, np.concatenate(concatenated_features, axis=0))

    print(f"Processed and saved features for batch {i//batch_size + 1}/{len(all_files)//batch_size}")


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Processed and saved features for batch 1/1034
Processed and saved features for batch 2/1034
Processed and saved features for batch 3/1034
Processed and saved features for batch 4/1034
Processed and saved features for batch 5/1034
Processed and saved features for batch 6/1034
Processed and saved features for batch 7/1034
Processed and saved features for batch 8/1034
Processed and saved features for batch 9/1034
Processed and saved features for batch 10/1034
Processed and saved features for batch 11/1034
Processed and saved features for batch 12/1034
Processed and saved features for batch 13/1034
Processed and saved features for batch 14/1034
Processed and saved features for batch 15/1034
Processed and saved features for batch 16/1034
Processed and saved features for batch 17/1034
Processed and saved features for batch 18/1034
Processed and saved features for batch 19/1034
Processed and saved features for batch 20/1034
Processed and saved features for batch 21/1034
Processed and saved fe